# Set up

In [12]:
import requests
import json, os

nn_id = 'nn910798'  # put some key value you want to test

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")
print("done")

done


# Net & Version Create

In [13]:
####(1) 네트워크 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/',
                     json={
                         "biz_cate": "MES",
                         "biz_sub_cate": "M60",
                         "nn_title" : "test",
                         "nn_desc": "test desc",
                         "use_flag" : "Y",
                         "dir": "purpose?",
                         "config": "N"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

####(2) 버전 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/version/',
                 json={
                     "nn_def_list_info_nn_id": "",
                     "nn_wf_ver_info": "test version info",
                     "condition": "1",
                     "active_flag": "Y"
                 })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


evaluation result : nn910798
evaluation result : nn910798


# Graph Flow 정의
CSV 데이터를 AutoEncoder 에 훈련시키는 Graph Flow 정의, Eval Node 는 정의하지 않음. 

In [14]:
# Work Flow 틀을 구성하도로고 지시한다. (정해진 틀을 강제로 생성)
resp = requests.post('http://' + url + '/api/v1/type/wf/target/init/mode/simple/' + nn_id +'/wfver/1/',
                     json={
                         "type": 'autoencoder_csv'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : autoencoder_csv


# CSV Data Upload 
한글 CSV 데이터를 업로드 한다. 

In [15]:
return_dict = {}
return_dict['test'] = open('../../data/seq2seq_friday.csv', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/datasrc/',
                     files = return_dict)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : ['1 file upload success']


# Data Node 정의
데이터 처리에 관한 정의 CSV 파일을 Pandas 를 통해서 읽고 HDF5 로 변환, 별도의 전처리는 지정하지 않음 

In [16]:
# (1) Train Data Node 의 속성을 정의
# 어디서 Source 를 어떻게 가지고 올것인지 파라메터 정의 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/datasrc/',
                     json={
                         "type": "csv",
                         "source_server": "local",
                         "source_sql": "all",
                     })

# 전처리는 어떤 것을 할지 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/pre/nnid/'+nn_id+'/ver/1/node/datasrc/',
                      json={
                          "preprocess":  "none",
                      })
# 전처리가 완료된 데이터는 어디에 저장을 할지 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/store/nnid/'+nn_id+'/ver/1/node/datasrc/',)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : /hoya_str_root/nn910798/1/datasrc


# Data Feeder 
Network 에서 훈련을 위해 원하는 형태로 데이터를 편집하여 제공하는 기능 수행 

In [17]:
# (2) Network 에 데이터를 Feed하는 Node 의 속성을 정의 
resp = requests.post('http://' + url + '/api/v1/type/wf/state/pre/detail/feed/src/frame/net/autoencoder/nnid/'+nn_id+'/ver/1/node/feed_train/',
                     json={
                         "encode_column" : "encode",
                         "encode_len" : 10,
                         "vocab_size" : 100,
                         "preprocess": "mecab",
                         "embed_type" : 'onehot'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'vocab_size': 100, 'embed_type': 'onehot', 'preprocess': 'mecab', 'encode_column': 'encode', 'encode_len': 10}


# AutoEncoder 정의 
n_hidden 의 경우 Encoder 부만 정의하면 Decoder부는 동일한 Vector 를 Revsere로 활용하도록 되어 있다. 

In [18]:
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/autoencoder/nnid/' + nn_id + '/ver/1/node/netconf_node/',
                     json={
                        "learning_rate" : 0.01,
                        "iter" : 10,
                        "batch_size" : 10,
                        "examples_to_show" : 10,
                        "n_hidden" : [200, 100, 10] 
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'model_path': '/hoya_model_root/nn910798/1/netconf_node', 'n_hidden': [200, 100, 10], 'examples_to_show': 10, 'iter': 10, 'learning_rate': 0.01, 'batch_size': 10}


# Train 을 시작한다. 

In [37]:
# Run All Workflow
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/1/')
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : [None, None, 'nn910798_1_netconf_node']


# Compressed Vector 를 구해보자 
해당 Vector 의 Size 는 "n_hidden" : [200, 100, 10]에서 정의한 마지막 Layer 의 사이즈가 될 것이다. 

In [59]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/autoencoder/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "드래곤은 무엇인가?" ,
                           "type" : "encoder"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : [[0.999107301235199, 6.752531044185162e-05, 0.0005148454220034182, 0.04103323817253113, 0.9974204301834106, 0.06846089661121368, 0.005738380830734968, 2.1049938368378207e-05, 0.003721178276464343, 0.07773767411708832]]


# 복원된 Vector 추출 
복원된 Vector는 사전 Filter 개념으로 이해하고 다른 네트워크의 인풋으로도 사용할 수 있다. 

In [60]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/autoencoder/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "드래곤은 무엇인가?" ,
                           "type" : "decoder"}
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : [[3.4912275737042364e-08, 5.4059677495388314e-05, 0.9998898506164551, 0.93865966796875, 0.2157174050807953, 0.9678962826728821, 0.04876554757356644, 0.14993269741535187, 0.001359722577035427, 0.002648558933287859, 0.659841775894165, 0.999457061290741, 0.7913259863853455, 0.9986515641212463, 0.9999909400939941, 0.9999992847442627, 0.9999982118606567, 0.8502862453460693, 0.0012591298436746001, 0.03365534543991089, 2.1912745069130324e-05, 0.9046458601951599, 0.32551440596580505, 0.967491865158081, 0.9998031258583069, 0.5872603058815002, 0.9998083710670471, 0.8350116610527039, 0.9813627004623413, 0.9983355402946472, 0.9999258518218994, 0.016223551705479622, 0.2772129774093628, 0.999776303768158, 0.9957996010780334, 1.0125398297589072e-09, 0.00012454975512810051, 0.9978713989257812, 2.2775178877054714e-05, 1.115569081555634e-09, 0.008903020061552525, 0.00010163749539060518, 0.8093627095222473, 0.0029364535585045815, 0.9933531284332275, 0.9999971389770508, 0.9999998807907